In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D,AveragePooling2D, Flatten, Dropout, concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score


In [3]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape and normalize images
x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

# Convert labels to one-hot encoded vectors
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)


11490434/11490434 [==============================] - 0s 0us/step


In [4]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, concatenate, Flatten, Dense, Dropout
from tensorflow.keras.models import Model

def inception_module(x, filters):
    tower_1 = Conv2D(filters[0], (1, 1), padding='same', activation='relu')(x)
    tower_1 = Conv2D(filters[1], (3, 3), padding='same', activation='relu')(tower_1)

    tower_2 = Conv2D(filters[2], (1, 1), padding='same', activation='relu')(x)
    tower_2 = Conv2D(filters[3], (5, 5), padding='same', activation='relu')(tower_2)

    tower_3 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    tower_3 = Conv2D(filters[4], (1, 1), padding='same', activation='relu')(tower_3)

    output = concatenate([tower_1, tower_2, tower_3], axis=3)
    return output

def create_inception_model():
    input_layer = Input(shape=(28, 28, 1))

    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(input_layer)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = Conv2D(64, (1, 1), padding='same', activation='relu')(x)
    x = Conv2D(192, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = inception_module(x, [64, 96, 128, 16, 32])
    x = inception_module(x, [128, 128, 192, 32, 96])
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = inception_module(x, [192, 96, 208, 16, 48])
    x = inception_module(x, [160, 112, 224, 24, 64])
    x = inception_module(x, [128, 128, 256, 24, 64])
    x = inception_module(x, [112, 144, 288, 32, 64])
    x = inception_module(x, [256, 160, 320, 32, 128])
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = inception_module(x, [256, 160, 320, 32, 128])
    x = inception_module(x, [384, 192, 384, 48, 128])

    x = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    x = Dropout(0.4)(x)
    x = Flatten()(x)
    output_layer = Dense(10, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# Create the Inception v1 model
inception_model = create_inception_model()
inception_model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 28, 28, 1)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 14, 14, 64)           3200      ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 7, 7, 64)             0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_1 (Conv2D)           (None, 7, 7, 64)             4160      ['max_pooling2d[0][0]']   

In [5]:
# Compile the model
inception_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
inception_model.fit(x_train, y_train, batch_size=128, epochs=5, validation_data=(x_test, y_test))


Epoch 1/5
469/469 [==============================] - 531s 1s/step - loss: 1.2395 - accuracy: 0.5239 - val_loss: 0.2528 - val_accuracy: 0.9283
Epoch 2/5
469/469 [==============================] - 509s 1s/step - loss: 0.1494 - accuracy: 0.9622 - val_loss: 0.0732 - val_accuracy: 0.9802
Epoch 3/5
469/469 [==============================] - 507s 1s/step - loss: 0.0917 - accuracy: 0.9783 - val_loss: 0.0591 - val_accuracy: 0.9840
Epoch 4/5
469/469 [==============================] - 505s 1s/step - loss: 0.0703 - accuracy: 0.9837 - val_loss: 0.0470 - val_accuracy: 0.9888
Epoch 5/5
469/469 [==============================] - 496s 1s/step - loss: 0.0582 - accuracy: 0.9869 - val_loss: 0.0560 - val_accuracy: 0.9873


In [6]:
# Evaluate the model and print the confusion matrix
y_pred = inception_model.predict(x_test)
y_pred = tf.argmax(y_pred, axis=1)
y_test = tf.argmax(y_test, axis=1)

# Compute the confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)
#print("Confusion Matrix:")
print(confusion_mat)


313/313 [==============================] - 15s 46ms/step
[[ 977    0    0    0    1    0    1    1    0    0]
 [   0 1128    0    4    0    0    0    3    0    0]
 [   1    1 1021    1    0    0    0    8    0    0]
 [   0    0    2 1001    0    5    0    2    0    0]
 [   0    0    1    0  975    0    0    2    0    4]
 [   0    0    0    4    0  887    1    0    0    0]
 [   3    3    0    1    5    8  937    0    1    0]
 [   0    1    2    1    0    0    0 1024    0    0]
 [   2    0    2   13    1    8    0    2  945    1]
 [   0    0    0    3    9    3    0   16    0  978]]


In [7]:
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Precision: 0.9874862635139611
Recall: 0.9873
F1-score: 0.9873052727574312
